In [3]:
import pandas as pd

In [4]:
df_train = pd.read_parquet('df_reponse_daily_train.parquet')
df = pd.read_csv('factor_return.csv')

In [5]:
df_filtered = df.filter(regex='^fr_21d_next_raw')

In [8]:
df_train

,stock_id,liq_weight,mktcap_weight,equal_weight,21d_next_raw,10d_next_raw,5d_next_raw,1d_next_raw,21d_next_mktAdj,10d_next_mktAdj,...,MOMENTUM,LEVERAGE,VALUE,EARNYLD,GROWTH,LTREV,STREV,industry_code,st_status,price
time,,,,,,,,,,,,,,,,,,,,,
2010-01-04 10:00:00,1,3.093088e+09,7.530677e+10,1.0,-0.109278,-0.105979,-0.054021,-0.047835,-0.047374,-0.127839,...,0.737709,0.557227,-0.306540,-0.655995,0.214168,0.000000,0.572045,28,0.0,24.250000
2010-01-04 10:00:00,2,4.916040e+09,1.032883e+11,1.0,-0.108716,-0.051546,-0.032802,-0.035614,-0.046811,-0.073406,...,-0.473381,0.360857,0.466091,0.250722,-0.004936,0.000000,1.424041,11,0.0,10.720141
2010-01-04 10:00:00,5,5.239416e+08,5.467715e+09,1.0,-0.090301,0.021739,-0.013378,-0.016722,-0.028396,-0.000120,...,1.227050,-0.792016,-1.179613,-0.736454,-0.944968,0.000000,-0.685017,11,1.0,5.980000
2010-01-04 10:00:00,6,6.517194e+08,5.685524e+09,1.0,-0.077609,-0.007136,-0.036574,-0.039251,-0.015705,-0.028996,...,0.396165,1.905245,0.353328,-0.193752,-0.887294,0.000000,1.657673,11,0.0,11.282970
2010-01-04 10:00:00,7,1.407934e+08,1.298457e+09,1.0,0.004274,0.096866,0.011396,-0.017094,0.066178,0.075007,...,0.604593,0.467104,-2.195040,-0.370742,-0.951803,0.000000,0.873514,20,1.0,7.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-11-30 10:00:00,603993,1.945218e+09,1.268403e+11,1.0,-0.043175,-0.047354,-0.094708,-0.036212,-0.018731,-0.030799,...,0.413714,1.035243,-0.735953,-0.342288,-0.516140,0.317234,-0.368143,12,0.0,23.175574
2017-11-30 10:00:00,603996,2.592166e+08,5.384691e+09,1.0,-0.010033,0.026756,0.008361,0.013935,0.014411,0.043311,...,-0.395947,0.238730,-1.154499,-0.699089,0.117570,0.775154,-0.585079,18,0.0,27.474264
2017-11-30 10:00:00,603997,1.910796e+08,7.434000e+09,1.0,-0.039831,-0.003390,-0.013559,-0.012712,-0.015386,0.013165,...,-0.286645,-1.106616,-1.327366,-0.184245,-0.084308,-0.330599,0.491384,14,0.0,18.072642


In [7]:
df_series = df_train.pivot(index='time', columns='stock_id')

KeyError: "None of ['time'] are in the columns"